<a href="https://colab.research.google.com/github/aashrithresearch/endometrial-cancer/blob/main/models/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets transformers accelerate

In [ ]:
from datasets import load_dataset

dataset = load_dataset('/content/drive/MyDrive/histopathological image dataset for ET (2)')

Resolving data files:   0%|          | 0/3302 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
splits = dataset["train"].train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [ ]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 2971
})

In [ ]:
train_ds.features

{'image': Image(mode=None, decode=True, id=None),
 'label': ClassLabel(names=['Endometrial Adenocarcinoma', 'Endometrial Hyperplasia', 'Endometrial Polyp', 'Normal Endometrium'], id=None)}

In [ ]:
train_ds[0]['label']

0

In [ ]:
id2label = {id:label for id, label in enumerate(train_ds.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

{0: 'Endometrial Adenocarcinoma',
 1: 'Endometrial Hyperplasia',
 2: 'Endometrial Polyp',
 3: 'Normal Endometrium'}

In [ ]:
id2label[train_ds[0]['label']]

'Endometrial Adenocarcinoma'

In [ ]:
from transformers import ViTImageProcessor

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

In [ ]:
train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)

In [ ]:
train_ds[:2]

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>],
 'label': [0, 2],
 'pixel_values': [tensor([[[ 0.4196,  0.4980,  0.6235,  ...,  0.3725,  0.5059,  0.6471],
           [ 0.3490,  0.4510,  0.5686,  ...,  0.3412,  0.4980,  0.6314],
           [ 0.3176,  0.3255,  0.4353,  ...,  0.3490,  0.5059,  0.6157],
           ...,
           [ 0.7647,  0.7804,  0.7725,  ...,  0.7490,  0.7569,  0.7490],
           [ 0.7647,  0.7804,  0.7569,  ...,  0.7647,  0.7255,  0.7020],
           [ 0.7725,  0.7804,  0.7490,  ...,  0.7412,  0.7176,  0.6863]],
  
          [[-0.4353, -0.4118, -0.3490,  ..., -0.4431, -0.3255, -0.0275],
           [-0.4902, -0.4431, -0.3882,  ..., -0.4980, -0.4196, -0.1373],
           [-0.4902, -0.5216, -0.4667,  ..., -0.5294, -0.5059, -0.2706],
           ...,
           [ 0.5216,  0.5216,  0.4353,  ..., -0.2471, -0.2235, -0.1765],
           [ 0.5216,  0.5059,  0.4196,  ..., -0.2314, -0

In [ ]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_dataloader = DataLoader(train_ds, collate_fn=collate_fn, batch_size=4)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k, v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                  id2label=id2label,
                                                  label2id=label2id)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

metric_name = "accuracy"

args = TrainingArguments(
    f"test-et",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels), precision_recall_fscore_support=precision_recall_fscore_support(predictions, labels, average='macro'))

In [ ]:
import torch

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Recall Fscore Support
1,No log,0.724023,0.728097,"(0.7119577859879584, 0.7248735073871686, 0.7170677505136802, None)"
2,0.874000,0.586812,0.797583,"(0.8038429960843754, 0.7827887258189661, 0.79092348176473, None)"
3,0.874000,0.560674,0.773414,"(0.776061402677782, 0.7834139665481578, 0.7726029314242115, None)"
4,0.593500,0.481282,0.812689,"(0.7971632773356911, 0.8265612475638859, 0.8040014803838254, None)"
5,0.593500,0.445510,0.833837,"(0.8348322175908383, 0.8260189416994488, 0.8289050146362535, None)"
6,0.480800,0.439200,0.842900,"(0.8364705696602248, 0.8324144167720122, 0.8332119421954906, None)"
7,0.389100,0.419403,0.854985,"(0.8444883899625278, 0.8496999899226847, 0.8459308793563278, None)"
8,0.389100,0.481561,0.815710,"(0.8373680318723422, 0.8081171177885004, 0.8153674496388574, None)"
9,0.346600,0.412058,0.864048,"(0.8529869794956001, 0.8613562091503267, 0.856844589726199, None)"
10,0.346600,0.412098,0.845921,"(0.8511566618247653, 0.8335836038961039, 0.8408542678613626, None)"


Trainer is attempting to log a value of "(0.7119577859879584, 0.7248735073871686, 0.7170677505136802, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.8038429960843754, 0.7827887258189661, 0.79092348176473, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.776061402677782, 0.7834139665481578, 0.7726029314242115, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.7971632773356911, 0.8265612475638859, 0.8040014803838254, None)" of type <class 'tuple'> for

TrainOutput(global_step=14900, training_loss=0.1921118470006341, metrics={'train_runtime': 9905.5257, 'train_samples_per_second': 14.997, 'train_steps_per_second': 1.504, 'total_flos': 1.1511641403950285e+19, 'train_loss': 0.1921118470006341, 'epoch': 50.0})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "(0.8942170855963959, 0.8852819683908046, 0.8883576134416493, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.45438602566719055,
 'eval_accuracy': 0.8912386706948641,
 'eval_precision_recall_fscore_support': (0.8942170855963959,
  0.8852819683908046,
  0.8883576134416493,
  None),
 'eval_runtime': 6.2722,
 'eval_samples_per_second': 52.773,
 'eval_steps_per_second': 13.233,
 'epoch': 50.0}

In [ ]:
trainer.save_model()

In [ ]:
trainer.save_metrics("eval", trainer.evaluate())
trainer.save_state()